# Syncing files to Dropbox

In this project, you will write a script that syncs your folder to your Dropbox account, checking each file or directory to see if it needs updating.

Go through [Accessing the Dropbox API](get_dropbox_api.ipynb) first to get a Dropbox API token which you will need to access the Dropbox API.

In [ ]:
# This code sets up the Dropbox API class so it can be used later.
from dropbox_api import DBApi, APIError, RateLimitError

token = 'paste your Dropbox API token here'

## 1 Try out the Dropbox API

Instantiate the Dropbox API:

    db = DBApi(token)

In [ ]:
db = DBApi(token)

We now have a few methods available to us:

- `db.list_folder()`
- `db.create_folder()`
- `db.upload_file()`
- `db.delete()`

And an attribute:

- `db.root`

**Task 1.1:** Use `help()` to explore the usage of each method.

In [ ]:
help(db.list_folder)

**Task 1.2:** Call `db.list_folder()` and examine the response from it.

In [ ]:
response = db.list_folder()

# Examine the object `response` (it might be empty if you are running this project for the first time)
response

**Question:** What type of object is `response`, and how is it structured?

## 2 Get file/directory listing from Dropbox

**Task 2.1:** Write program code below to get a list of files and directories from Dropbox using the Dropbox API.

In [ ]:
# Write code below to get a list of entries, each entry being a file or
# directory in Dropbox.
# Store it in the variable `db_entries`
# By this point you shouldn't have anything in your Dropbox app folder,
# So don't be surprised to see an empty list!

db_entries = 



## 3 Get file/directory listing from local directory using `os`



**Task 3.2:** Use `os.listdir()` to generate a list of files and directories in your JupyterHub home directory. The full path of your home directory is `'/home/jupyter-<your_username>'`. So if your username is `anewuser`, your home directory will be at `'/home/jupyter-anewuser'`.

Note that `os.listdir()` will not list the contents of subdirectories for you; you will have to list them recursively.

In [ ]:
# Write code below to get a list of entries, each entry being a file or
# directory in your current directory.

import os

if os.path.isfile('local_entries.csv')
local_entries = 



**Task 3.3 (optional):** There are many files and folders in the listing that can be ignored. Edit your code above to ignore the following files and directories.

- `.ipynb_checkpoints`
- `__pycache__`
- `.ipython`
- `.bash_history`
- `.bashrc`
- `.local`
- `.npm`
- `.conda`
- `.cache`
- `.jupyter`
- `.config`
- `.profile`
- `.bash_logout`

## 4 Compare Dropbox and local entries

Nowe we try to sync the Dropbox entries in `db_entries` and the local entries in `local_entries`.

**Task 4.1 (optional):** Look at the format of `db_entries` and `local_entries` and think about how you would compare the two listings. Is the current format easy to compare? You may update your code from **Task 2.1** and **Task 3.2** to make them easier to compare.

### (A) Syncing algorithm

The general algorithm for mirroring your current directory on Dropbox (one-way sync) is as follows:

1. If there are any entries in `db_entries` that are not in `local_entries`, **delete** them.
2. If there are any entries in `local_entries` that are not in `db_entries`:
    - If the entry is a file, **upload** it.
    - If the entry is a directory, **create** it.
3. If there are any entries that are both in `db_entries` and `local_entries`:
    - If the entries are files, compare them to see if they have changed.
    - If the entries are directories, ignore them.
    
On what basis shall we compare the files in Step 3? We will look into that later. For now, carry out step 1 and 2.

**Task 4.2:** Try out the Dropbox `db.delete()`, `db.create_folder()` and `db.upload_file()` methods, and observe the response you get from each method.

1. Create a folder with `db.create_folder()`.
2. Upload a file into this folder with `db.upload_file()`.
3. Delete the file and folder with `db.delete()`.

After each step, verify that it was correctly executed by checking your Dropbox account. The folders and files should be created in `App/<your app name>`.

In [ ]:
# Test the dropbox API methods here



**Task 4.3:** Implement the one-way sync algorithm to sync your JupyterHub home directory to your Dropbox folder.

Before you embark on this task, drop me a message and I will back up your home directory for you. Just in case.

In [ ]:
# Write your one-way sync program here



### (B) Checking files for changes with `rev`

Notice that in the response from `db.upload_file()` (or `db.list_folder()`), files carry some additional information. There is a `content_hash` attribute and a `rev` attribute.

The `content_hash` is used to verify that a file has been uploaded correctly. This hash should match the hash of the file generated locally. the algorithm for doing so can be found [in Dropbox's documentation](https://www.dropbox.com/developers/reference/content-hash). **We will not do this hashing for this project.**

The `rev` (stands for **revision**) is generated each time the file is changed on Dropbox. By storing a copy of the `rev` and comparing it to the `rev` from Dropbox, we can easily know whether there have been any changes to the file on Dropbox.

To carry out step 3, therefore, we need to store the `rev` locally and compare it to the one returned from Dropbox.

The one-way sync algorithm with rev is as follows:

1. If there are any entries in `db_entries` that are not in `local_entries`, delete them.
2. If there are any entries in `local_entries` that are not in `db_entries`:
    - If the entry is a file, upload it **and store the `rev`**.
    - If the entry is a directory, create it.
3. If there are any entries that are both in `db_entries` and `local_entries`:
    - If the entries are files, compare **their `rev`s** to see if they have changed.
    - If the entries are directories, ignore them.

**Task 4.4:** Implement the syncing algorithm. You may have to edit your code from **Task 3.2** to store the `rev` in `local_entries`.

It is recommended that you initialise any `rev`s for local file entries to `None` rather than `0` or `False`, to avoid invisible errors.

In [ ]:
# Delete entries from db_entries that are not in local_entries
# Upload files from local_entries that are not in db_entries
# For files in both local_entries and db_entries, compare their revs,
# upload the file if the rev is different



## 5 Store the results to a file

The next time we sync the folder contents again, we will need to know the previous `rev`s for each file. That means we need to store them somewhere ... the easiest and most straightforward way is probably as a CSV file.

**Task 5.1:** Write a function, `export_csv()`, that will write `local_entries` and their `rev`s (where necessary) to a CSV file.

You should decide an appropriate format to store the entries and their `rev`s before writing your code.

In [ ]:
# Export local_entries to a CSV file

def export_csv(filename,entries):
    '''Store (current) local_entries with rev and mtime to CSV.'''
    # Write your code here

The next time you sync the files, you wouldn't want to generate the `local_entries` from scratch: there wouldn't be any `rev`s to check!

You would have to load it from the CSV file created previously.

But you would also need to check it against the **most current** file/directory listing to see if anything has changed. But how would we do that, since `os.listdir()` doesn't generate `rev`s for us to check?

Whenever a file is modified, the operating system updates its **modified date/time**. You can get the **modified date/time** for a file using `os.path.getmtime()`.

Hmm, deeper and deeper ... now we have to store an attribute for the **modified date/time**, `mtime`, so that we can check it against the latest listing.

**Task 5.2:** Write a function, `import_csv()`, that will read `local_entries`, their`rev`s, and their `mtimes` from a CSV file. You will also need to update your code from Task 5.1 to export the `mtime`s for each file.

In [ ]:
# Import local_entries from a CSV file

def import_csv(filename):
    '''Read in (previous) local_entries, rev, and mtime from CSV.'''
    # Write your code here

**Task 5.3:** Update your code from **Task 3.2** and **Task 5.1**.

Task 3.2 must read `local_entries` from `'local_entries.csv'` first. Then it must compare the `mtime` of files in `local_entries` to the current file & directory listing.
  - Where the `mtime` is the same, do nothing
  - If the `mtime` is different, set the `rev` to `None` so that it will be uploaded to Dropbox

# 6 Put it all together

Now that you have all the pieces written, see if you can work out the logic of the program. What should happen first, and what should happen next?

Put them in the right order in the code cell below, and you should have a program taht successfully copies your JupyterHub home directory to Dropbox.

In [ ]:
# Arrange the code you have written above in the correct order
# to sync your JupyterHub home directory to Dropbox

